In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binance import Client
APIKEY = ""
SECRETKEY = ""

client = Client(APIKEY, SECRETKEY)
print("logged in")



def get_1m_data(symbol, time, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, 
                                                    time,
                                                    lookback))

    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High','Low','Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame



symbol1 = "BTCBUSD"
btc = get_1m_data(symbol1, client.KLINE_INTERVAL_5MINUTE, lookback='40000 min ago')
btc.columns = [c.lower() for c in btc.columns]
btc.to_csv("btc.csv")

logged in


In [26]:
import ta
from ta.volatility import BollingerBands
def apply_ta(df):
    # df['%k'] = ta.momentum.stoch(df.high,df.low,df.close, window=14,smooth_window=3)
    # df['%D'] = df['%k'].rolling(3).mean()
    df['RSI']= ta.momentum.rsi(df.close, window=14)
    df['macd']= ta.trend.macd_diff(df.close)
    df['75_MA'] = ta.trend.sma_indicator(df['close'], window=75)
    df['21_MA'] = ta.trend.sma_indicator(df['close'], window=21)
    df['14_MA'] = ta.trend.sma_indicator(df['close'], window=14)
    df['7_MA'] = ta.trend.sma_indicator(df['close'], window=7)
    df['3_MA'] = ta.trend.sma_indicator(df['close'], window=3)
    df['ATR'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=100).average_true_range()
    indicator_bb = BollingerBands(close=df["close"], window=20, window_dev=2)
    df['Boll']= indicator_bb.bollinger_wband()
    df.dropna(inplace=True)
    return df
    

def calculate_streak(df):
    streak = np.zeros(df.shape[0])
    print(df.shape[0])
    for i in range(1, df.shape[0]):
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            streak[i] = streak[i - 1] + 1
        else:
            streak[i] = 0
    return streak

def calculate_neg_streak(df):
    streak = np.zeros(df.shape[0])
    print(df.shape[0])
    for i in range(1, df.shape[0]):
        if df['close'].iloc[i] < df['close'].iloc[i-1]:
            streak[i] = streak[i - 1] + 1
        else:
            streak[i] = 0
    return streak



def get_model_data(df):   
    df["gap"] = ((df["close"] - df["open"])/df["close"])
    df["next"] = df["close"].shift(-1)
    df["target"] = (df["next"] > df["close"]).astype(int)
    df["3MA_diff"] = (df["3_MA"] - df["close"])/df["close"] 
    df["7MA_diff"] = (df["7_MA"] - df["close"])/df["close"]   
    df["14MA_diff"] = (df["14_MA"]- df["close"])/df["close"] 
    df["21MA_diff"] = (df["21_MA"] - df["close"])/df["close"] 
    df["75MA_diff"] =  (df["75_MA"] - df["close"])/df["close"] 
    df["condition1"] = ((df["3MA_diff"] > df["21MA_diff"]) & (df["21MA_diff"] > df["75MA_diff"])).astype(int)
    df["streak"] = calculate_streak(df)
    df["neg_streak"] = calculate_neg_streak(df)
    return df


btc = pd.read_csv("btc.csv")
btc = apply_ta(btc)
btc = get_model_data(btc)
pd.set_option('display.max_columns', None)


7926
7926


In [27]:
btc

,Time,open,high,low,close,volume,RSI,macd,75_MA,21_MA,14_MA,7_MA,3_MA,ATR,Boll,gap,next,target,3MA_diff,7MA_diff,14MA_diff,21MA_diff,75MA_diff,condition1,streak,neg_streak
74,2023-04-04 05:20:00,27920.70,27943.94,27914.01,27921.73,25.51704,62.199595,7.842647,27851.260400,27865.613810,27852.210000,27865.431429,27901.496667,0.000000,0.525084,0.000037,27911.39,0,-0.000725,-0.002016,-0.002490,-0.002010,-0.002524,1,0.0,0.0
75,2023-04-04 05:25:00,27921.73,27928.87,27911.37,27911.39,16.53720,59.423317,8.945321,27851.443733,27866.156190,27854.045714,27876.064286,27917.940000,0.000000,0.524747,-0.000370,27921.57,1,0.000235,-0.001266,-0.002055,-0.001621,-0.002148,1,0.0,1.0
76,2023-04-04 05:30:00,27911.39,27921.57,27911.38,27921.57,11.38049,61.256824,9.859764,27851.464667,27866.623810,27857.612143,27889.068571,27918.230000,0.000000,0.550213,0.000365,27938.06,1,-0.000120,-0.001164,-0.002291,-0.001968,-0.002511,1,1.0,0.0
77,2023-04-04 05:35:00,27921.56,27945.98,27899.19,27938.06,33.17406,64.087615,10.988508,27851.560267,27868.976190,27862.570714,27903.165714,27923.673333,0.000000,0.594902,0.000591,27919.20,0,-0.000515,-0.001249,-0.002702,-0.002473,-0.003096,1,2.0,0.0
78,2023-04-04 05:40:00,27938.06,27938.06,27907.13,27919.20,32.42983,58.796247,9.911401,27851.902533,27871.087143,27869.657143,27913.530000,27926.276667,0.000000,0.614047,-0.000676,27915.89,0,0.000253,-0.000203,-0.001775,-0.001723,-0.002410,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2023-05-01 17:25:00,28320.60,28322.66,28185.96,28196.69,84.95525,40.945369,12.049222,28429.365600,28249.590476,28273.728571,28333.920000,28290.080000,53.617270,1.139545,-0.004394,28233.46,1,0.003312,0.004867,0.002732,0.001876,0.008252,0,0.0,3.0
7996,2023-05-01 17:30:00,28196.69,28242.75,28184.17,28233.46,57.06875,44.679673,6.266212,28425.331733,28251.281905,28281.546429,28313.894286,28251.693333,53.666898,1.133866,0.001302,28266.37,1,0.000646,0.002849,0.001703,0.000631,0.006796,0,1.0,0.0
7997,2023-05-01 17:35:00,28233.46,28287.81,28231.88,28266.37,33.60970,47.857749,4.543443,28421.713333,28253.866190,28290.914286,28295.520000,28232.173333,53.689529,1.113603,0.001164,28290.14,1,-0.001210,0.001031,0.000868,-0.000442,0.005496,0,2.0,0.0
7998,2023-05-01 17:40:00,28266.37,28341.18,28266.37,28290.14,39.52201,50.088083,4.862217,28418.088800,28258.770476,28295.902857,28287.940000,28263.323333,53.900734,1.118126,0.000840,28314.99,1,-0.000948,-0.000078,0.000204,-0.001109,0.004523,0,3.0,0.0


In [28]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
model1 = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=50, random_state=1)
model2 = RandomForestClassifier(n_estimators=400, max_depth=10, min_samples_split=40, random_state=2)
voting_clf = VotingClassifier(estimators=[('rf1', model1), ('rf2', model2)],  voting='hard')

In [29]:
predictors = ["open", "close", "low", "high",
                "gap" , "macd" ,"14_MA", "Boll", "3MA_diff", "7MA_diff", "14MA_diff", 
              "21MA_diff",  "RSI", "streak", "ATR", "neg_streak"]
train = btc.iloc[:7000]
test = btc.iloc[7000:]
voting_clf.fit(train[predictors], train["target"])
print("")

In [30]:
test["target"].value_counts(1)

1    0.50432
0    0.49568
Name: target, dtype: float64

In [31]:
from sklearn.metrics import precision_score, accuracy_score

preds = voting_clf.predict(test[predictors])
preds = pd.Series(preds, index=test.index, name="predictions")
test = pd.concat([test, preds], axis=1)
precision = precision_score(test["target"], preds)
accuracy = accuracy_score(test["target"], preds)
print(f"Precision: {precision}\n Accuracy: {accuracy}")
#0.562
#.542

Precision: 0.5714285714285714
 Accuracy: 0.5464362850971922


In [42]:
test

,Time,open,high,low,close,volume,RSI,macd,75_MA,21_MA,14_MA,7_MA,3_MA,ATR,Boll,gap,next,target,3MA_diff,7MA_diff,14MA_diff,21MA_diff,75MA_diff,condition1,streak,neg_streak,predictions
7074,2023-04-28 12:40:00,29331.93,29343.15,29299.64,29339.70,32.63337,55.917557,7.157092,29338.323333,29295.100000,29293.484286,29317.621429,29323.520000,53.177201,0.420099,0.000265,29244.56,0,-0.000551,-0.000753,-0.001575,-0.001520,-0.000047,0,2.0,0.0,0
7075,2023-04-28 12:45:00,29339.70,29344.99,29240.43,29244.56,61.56431,43.714790,1.150447,29335.729333,29293.790952,29291.277143,29312.664286,29305.393333,53.691029,0.446415,-0.003253,29300.45,1,0.002080,0.002329,0.001597,0.001683,0.003117,0,0.0,1.0,1
7076,2023-04-28 12:50:00,29244.55,29310.35,29223.61,29300.45,75.46177,50.542816,0.874801,29333.804267,29294.575714,29291.312143,29315.584286,29294.903333,54.021519,0.443800,0.001908,29241.44,0,-0.000189,0.000517,-0.000312,-0.000200,0.001138,0,1.0,0.0,0
7077,2023-04-28 12:55:00,29300.44,29300.45,29238.04,29241.44,32.53366,44.416227,-3.128961,29331.090000,29291.258571,29290.372857,29300.244286,29262.150000,54.105404,0.470092,-0.002018,29274.25,1,0.000708,0.002011,0.001673,0.001704,0.003066,0,0.0,1.0,1
7078,2023-04-28 13:00:00,29241.12,29285.45,29214.92,29274.25,37.18622,48.177566,-3.392679,29328.564133,29290.712381,29288.584286,29290.180000,29272.046667,54.269650,0.458998,0.001132,29277.14,1,-0.000075,0.000544,0.000490,0.000562,0.001855,0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2023-05-01 17:25:00,28320.60,28322.66,28185.96,28196.69,84.95525,40.945369,12.049222,28429.365600,28249.590476,28273.728571,28333.920000,28290.080000,53.617270,1.139545,-0.004394,28233.46,1,0.003312,0.004867,0.002732,0.001876,0.008252,0,0.0,3.0,1
7996,2023-05-01 17:30:00,28196.69,28242.75,28184.17,28233.46,57.06875,44.679673,6.266212,28425.331733,28251.281905,28281.546429,28313.894286,28251.693333,53.666898,1.133866,0.001302,28266.37,1,0.000646,0.002849,0.001703,0.000631,0.006796,0,1.0,0.0,1
7997,2023-05-01 17:35:00,28233.46,28287.81,28231.88,28266.37,33.60970,47.857749,4.543443,28421.713333,28253.866190,28290.914286,28295.520000,28232.173333,53.689529,1.113603,0.001164,28290.14,1,-0.001210,0.001031,0.000868,-0.000442,0.005496,0,2.0,0.0,1
7998,2023-05-01 17:40:00,28266.37,28341.18,28266.37,28290.14,39.52201,50.088083,4.862217,28418.088800,28258.770476,28295.902857,28287.940000,28263.323333,53.900734,1.118126,0.000840,28314.99,1,-0.000948,-0.000078,0.000204,-0.001109,0.004523,0,3.0,0.0,0


In [9]:
from sklearn.metrics import precision_score, accuracy_score
model1 = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=50, random_state=1)
predictors = ["open", "close", "low", "high",
                "gap" , "macd" ,"14_MA", "Boll", "3MA_diff", "7MA_diff", "14MA_diff", 
              "21MA_diff",  "RSI", "streak", "ATR", "neg_streak"]
train = btc.iloc[:7000]
test = btc.iloc[7000:]
model1.fit(train[predictors], train["target"])
print("")
preds2 = model1.predict(test[predictors])
preds2 = pd.Series(preds2, index=test.index, name="predictions")
test = pd.concat([test, preds2], axis=1)
precision = precision_score(test["target"], preds2)
accuracy = accuracy_score(test["target"], preds2)
print(f"Precision: {precision}\n Accuracy: {accuracy}")

Precision: 0.5768025078369906
 Accuracy: 0.550755939524838


In [13]:
importances = model1.feature_importances_

# If you want to display each feature with its importance
for feature, importance in zip(predictors, importances):
    print(f'The importance of {feature} is: {importance}')

The importance of open is: 0.048509339539426315
The importance of close is: 0.04821119368158907
The importance of low is: 0.05216410397435374
The importance of high is: 0.04947734238513473
The importance of gap is: 0.09275842977740391
The importance of macd is: 0.0672899084892567
The importance of 14_MA is: 0.05192066468900337
The importance of Boll is: 0.06951545051898375
The importance of 3MA_diff is: 0.07784907238035473
The importance of 7MA_diff is: 0.08173461998831504
The importance of 14MA_diff is: 0.07836814398062004
The importance of 21MA_diff is: 0.08731582460774921
The importance of RSI is: 0.10033251946488578
The importance of streak is: 0.011793282297978566
The importance of ATR is: 0.06752721335997737
The importance of neg_streak is: 0.015232890864967716


In [40]:
def back_investment_comp(model, train, test, investment, predictors):
    trades = 0
    balance = investment   
    commission_rate = 0
    model.fit(train[predictors], train["target"])
    prediction = model.predict(test[predictors])
    prediction = pd.Series(prediction, index=test.index, name="Predictions")
    combined = pd.concat([test, prediction], axis=1)
    balance_list = []
    for i in range(test.shape[0]-1):
        if (combined["Predictions"].iloc[i] == 1):
            if combined["low"].iloc[i+1] < combined["open"].iloc[i+1]*0.99999:
                balance = balance*0.99999
            
            else:
                balance = balance * (1 + combined["gap"].iloc[i+1]) 
                balance = balance * (1 - commission_rate)
            trades += 1
            balance_list.append(balance)
    
    roi = balance/investment
    return balance, trades, roi, balance_list


def back_investment_with_short_comp(model, train, test, investment, predictors):
    trades = 0
    balance = investment   
    model.fit(train[predictors], train["target"])
    prediction = model.predict(test[predictors])
    prediction = pd.Series(prediction, index=test.index, name="Predictions")
    combined = pd.concat([test, prediction], axis=1)
    balance_list = []
    print(f"Test shape: {test.shape[0]}")
    for i in range(combined.shape[0]-1):
        if (combined["Predictions"].iloc[i] == 1):
            if combined["low"].iloc[i+1] < combined["open"].iloc[i+1]*0.99999:
                balance = balance*0.99999
            
            else:
                balance = balance * (1 + combined["gap"].iloc[i+1])
            trades += 1
            balance_list.append(balance)
    
        else:
            if combined["high"].iloc[i+1] > combined["open"].iloc[i+1]*1.00005:
                balance = balance*0.99995

            else:
                balance = balance * (1 + (combined["gap"].iloc[i+1]*-1))
            balance_list.append(balance)
            trades += 1

    roi = balance/investment
    return balance, trades, roi, balance_list  

def back_short_commission(model, train, test, investment, predictors, commission_rate=0.00075, short_leverage=4):
    trades = 0
    balance = investment   
    model.fit(train[predictors], train["target"])
    prediction = model.predict(test[predictors])
    prediction = pd.Series(prediction, index=test.index, name="Predictions")
    combined = pd.concat([test, prediction], axis=1)
    balance_list = []
    print(f"Test shape: {test.shape[0]}")
    for i in range(combined.shape[0]-1):
        if (combined["Predictions"].iloc[i] == 1):
            if combined["low"].iloc[i+1] < combined["open"].iloc[i+1]*0.99999:
                balance = balance*0.99999
            
            else:
                balance = balance * (1 + combined["gap"].iloc[i+1])
            balance = balance * (1 - commission_rate)
            trades += 1
            balance_list.append(balance)
    
        else:
            if combined["high"].iloc[i+1] > combined["open"].iloc[i+1]*1.00005:
                balance = balance * (1 - (1 - 0.99995) * short_leverage)

            else:
                balance = balance * (1 + (combined["gap"].iloc[i+1]*-1*short_leverage))
            balance = balance * (1 - commission_rate)
            balance_list.append(balance)
            trades += 1

    roi = balance/investment
    return balance, trades, roi, balance_list

In [41]:
balance, trades, roi, balance_list = back_investment_comp(voting_clf, train, test, 1000,  predictors)
print("balance:", balance, "\nTrades:",trades, "\nROI:", roi*100,"%")

balance: 1052.3475959112411 
Trades: 329 
ROI: 105.23475959112412 %


In [9]:
balance2, trades2, roi2, balance_list2 = back_investment_with_short_comp(voting_clf, train, test, 1000,  predictors)
print("balance:", balance2, "\nTrades:",trades2, "\nROI:", roi2*100,"%")

Test shape: 1926
balance: 1309.5009032744513 
Trades: 1925 
ROI: 130.95009032744514 %


In [10]:
balance3, trades3, roi3, balance_list3 = back_short_commission(voting_clf, train, test, 1000,  predictors)
print("balance:", balance3, "\nTrades:",trades3, "\nROI:", roi3*100,"%")

Test shape: 1926
balance: 503.2034128854239 
Trades: 1925 
ROI: 50.320341288542394 %
